# LINE Messaging API

# Install Dependency

In [1]:
#!pip install requests

# Global vars

In [ ]:
# we need to find a way to keep this in the memory.
#Since the code is always running and only sends messages at some intervals, it shouldn't be that hard to do
PLANT_NAME = "Eustoma russellianum"  # Example plant name

## Broadcast Message

In [15]:
import requests
import os
from typing import Dict, Any, Optional, List

class PlantitaBot:
    def __init__(self, channel_access_token: str):
        """Initialize the Plantita LINE bot

        Args:
            channel_access_token (str): Your LINE channel access token
        """
        self.channel_access_token = channel_access_token
        self.headers = {
            'Authorization': f'Bearer {channel_access_token}',
            'Content-Type': 'application/json'
        }
        self.base_url = 'https://api.line.me/v2/bot'

    def get_followers(self) -> List[str]:
        """Get list of user IDs who are friends with your LINE Official Account

        Returns:
            List[str]: List of user IDs
        """
        user_ids = []
        next_cursor = None

        while True:
            endpoint = f'{self.base_url}/followers/ids'
            if next_cursor:
                endpoint += f'?start={next_cursor}'

            response = requests.get(endpoint, headers=self.headers)
            data = response.json()

            if 'userIds' in data:
                user_ids.extend(data['userIds'])

            if 'next' not in data:
                break

            next_cursor = data['next']

        return user_ids

    def broadcast_message(self, message: str) -> Dict[str, Any]:
        """Broadcast a message to all friends of your LINE Official Account

        Args:
            message (str): The message to broadcast

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        endpoint = f'{self.base_url}/message/broadcast'

        data = {
            'messages': [{
                'type': 'text',
                'text': message
            }]
        }

        response = requests.post(endpoint, headers=self.headers, json=data)
        return response.json()




    def broadcast_plant_status(self,
                               temperature: float,
                               humidity: float,
                               light_level: float,
                               status: str = "normal") -> Dict[str, Any]:
        """Broadcast plant status to all friends

        Args:
            temperature (float): Current temperature reading
            humidity (float): Current humidity reading
            light_level (float): Current light level reading
            status (str): Overall status of the plant ("normal", "warning", "critical")

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        message = (
            f"🌿 Plantita Status Update 🌿\n\n"
            f"Temperature: {temperature}°C\n"
            f"Humidity: {humidity}%\n"
            f"Light Level: {light_level}%\n\n"
            f"Status: {status.upper()}"
        )

        if status.lower() != "normal":
            message += "\n\n⚠️ Action needed! Please check your plant."

        return self.broadcast_message(message)

    def send_message(self, user_id: str, message: str) -> Dict[str, Any]:
        """Send a message to a specific user

        Args:
            user_id (str): The LINE user ID to send the message to
            message (str): The message to send

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        endpoint = f'{self.base_url}/message/push'

        data = {
            'to': user_id,
            'messages': [{
                'type': 'text',
                'text': message
            }]
        }

        response = requests.post(endpoint, headers=self.headers, json=data)
        return response.json()

    def send_plant_status(self,
                          user_id: str,
                          temperature: float,
                          humidity: float,
                          light_level: float,
                          status: str = "normal") -> Dict[str, Any]:
        """Send plant status to a specific user

        Args:
            user_id (str): The LINE user ID to send the message to
            temperature (float): Current temperature reading
            humidity (float): Current humidity reading
            light_level (float): Current light level reading
            status (str): Overall status of the plant ("normal", "warning", "critical")

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        message = (
            f"🌿 Plantita Status Update 🌿\n\n"
            f"Temperature: {temperature}°C\n"
            f"Humidity: {humidity}%\n"
            f"Light Level: {light_level}%\n\n"
            f"Status: {status.upper()}"
        )

        if status.lower() != "normal":
            message += "\n\n⚠️ Action needed! Please check your plant."

        return self.send_message(user_id, message)

## Example Usage of Broadcast

In [4]:
# Example usage:
if __name__ == "__main__":
    # Initialize the bot with your channel access token
    CHANNEL_ACCESS_TOKEN = "HYIfkCHmidAfgMc2KcXBLUXzpNvsvJ/t5k8VwU2+ekWovndds7h/5h2Qq+SZkoYjNCPuLg0BHANjAnX81xjbfT7imjJ1s+gYZf6XU4ttM4kuFTkChGSHDZUVKzFpTgesDxTWmILCYw0vgPz/bse7TQdB04t89/1O/w1cDnyilFU="  # Replace with your actual token

    bot = PlantitaBot(CHANNEL_ACCESS_TOKEN)

    # Example: Get list of followers
    followers = bot.get_followers()
    print(f"Number of followers: {len(followers)}")
    print("Follower IDs:", followers)

    # Example: Broadcast a simple message to all followers
    bot.broadcast_message("Hello everyone from Plantita! 🌱")

    # Example: Broadcast plant status to all followers
    bot.broadcast_plant_status(
        temperature=25.5,
        humidity=65.0,
        light_level=80.0,
        status="warning"
    )

Number of followers: 0
Follower IDs: []


## Example Usage of Direct Message
**Still Not Working** Need to setup Webhook Server.

In [ ]:
# Initialize the bot
bot = PlantitaBot("HYIfkCHmidAfgMc2KcXBLUXzpNvsvJ/t5k8VwU2+ekWovndds7h/5h2Qq+SZkoYjNCPuLg0BHANjAnX81xjbfT7imjJ1s+gYZf6XU4ttM4kuFTkChGSHDZUVKzFpTgesDxTWmILCYw0vgPz/bse7TQdB04t89/1O/w1cDnyilFU=")

# Send a plant status update
# bot.send_plant_status(
#     user_id="USER_ID",
#     temperature=25.5,
#     humidity=65.0,
#     light_level=80.0,
#     status="warning"
# )

{'message': "The property, 'to', in the request body is invalid (line: -, column: -)"}

## GROQ Implementation

# Install Dependency

In [6]:
#!pip install groq

# Demonstration

In [7]:
import os

from groq import Groq

client = Groq(
    api_key="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",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": '''You are Plantita, an expert plant care advisor with a warm, caring personality like a concerned aunt.
Current readings for Monstera Deliciosa:
        temperature=27.5,
        humidity=55,
        light_level=40,

Ideal conditions for this plant:
- Temperature: 20°C to 25°C
- Humidity: 60% to 80%
- Light Level: 50% to 80%

Please analyze these conditions and provide:
1. A caring, conversational assessment of the plant's current environment
2. Specific recommendations for improvement if needed
3. Any potential risks to the plant's health based on these conditions
4. A simple action plan for the plant owner

Keep your response friendly and encouraging, like a knowledgeable aunt giving advice about their beloved plants.
Limit your response to fit a notification on a messaging app, focus only on what the user needed to do and keep it to three to 5 sentences''',
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Sweetie, I've taken a look at your Monstera Deliciosa's environment, and I'm happy to report that it's mostly doing well! However, I do see a few areas where we can make some adjustments to help your plant thrive. The temperature is a bit on the warm side, and the humidity is a tad low. To keep your plant happy, I recommend keeping an eye on the temperature and making sure it stays between 20-25°C. Additionally, you can try increasing the humidity around your plant by placing it on a tray with pebbles and water or using a humidifier.


# Integrate sensor information with Groq

In [8]:
#!pip install pyserial pysimplegui 



# Simulate Arduino data on PC if we forgot to carry the Arduino

# Actual arduino data

In [9]:
import serial
import time


# On Linux, it might be something like '/dev/ttyUSB0' or '/dev/ttyACM0'
# On Mac, it could be '/dev/tty.usbmodem*'
serial_port ='COM5'  
baud_rate = 9600  # Must match the baud rate set in the Arduino code

# Open the serial connection
ser = serial.Serial(serial_port, baud_rate)
time.sleep(2)  # Allow time for the connection to establish

print("Collecting data from Arduino...")

try:
    while True:
        # Read a line from the serial data
        line = ser.readline().decode('utf-8').strip()

        # Check if the line contains valid sensor data
        if line:
            print(line)  # Print the line or process it as needed

        time.sleep(1)  # Delay for readability, adjust as needed

except KeyboardInterrupt:
    print("Data collection stopped.")

finally:
    ser.close()  # Always close the serial connection when done


SerialException: could not open port 'COM5': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

# Get PlantID


# Getting Plant ID from a picture

In [8]:
import requests
import base64
#from google.colab import userdata
# Load the image and encode it in base64
image_path = "plant.jpg" #! Need to substitute for ngrok link

def identify_plant(image_path):
    # Define the endpoint and API key
    url = "https://api.plant.id/v2/identify"
    #api_key = userdata.get('plantid_key')  # Replace with your actual API key
    api_key = "V3bNS9Yrc6yWsY2pgevJnjNoYtnhIw6Uvh9PxvuNwiyTyYf5Tr"
    
    # need to add this:
    #https://plant.id/api/v3/kb/plants/:access_token?details=common_names,
    # url,description,taxonomy,rank,gbif_id,inaturalist_id,image,synonyms,edible_parts,watering,propagation_methods


    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode("utf-8")

    # Prepare the headers and data payload
    headers = {
        "Content-Type": "application/json",
        "Api-Key": api_key
    }

    data = {
        "images": [base64_image],
        "organs": ["leaf"],  # Specify which part of the plant you are identifying (leaf, flower, etc.)
        "include-related-images": False,  # Optional: Include related images in response
        "include-plant-details": ["common_names", "url", "wiki_description"]  # Specify the plant details you need
    }

    # Send POST request
    response = requests.post(url, headers=headers, json=data)

    # Initialize variable to store the plant name with the highest probability
    plant_name_with_highest_probability = None

    # Check the response
    if response.status_code == 200:
        results = response.json()

        #! Need to fix the probelem with the results and getting the details
        # water = results["watering"]
        # best_watering = results["best_watering"]
        # best_light_condition = results["best_light_condition"]
        # best_soil_type = results["best_soil_type"]

        # print(f"Watering: {water['min']} to {water['max']}")
        # print("Best Watering Practices:", best_watering)
        # print("Best Light Conditions:", best_light_condition)
        # print("Best Soil Types:", best_soil_type)
        # print("Possible plant identifications:")

        # Find the suggestion with the highest probability
        highest_probability = 0
        for suggestion in results["suggestions"]:
            probability = suggestion["probability"]

            if probability > highest_probability:
                highest_probability = probability
                plant_name_with_highest_probability = suggestion["plant_name"]

            # Print suggestion details
            print(f"Species: {suggestion['plant_name']}, Probability: {probability * 100:.2f}%")

            # Additional details (if requested)
            if "common_names" in suggestion:
                print("Common Names:", ", ".join(suggestion["common_names"]))
            if "url" in suggestion:
                print("More Info:", suggestion["url"])
            if "wiki_description" in suggestion and "value" in suggestion["wiki_description"]:
                print("Description:", suggestion["wiki_description"]["value"])
                print("Wiki:",suggestion["wiki_description"])
            print("\n")

        # Print the plant with the highest probability
        print(f"Plant with the highest probability: {plant_name_with_highest_probability}")
        return plant_name_with_highest_probability

    else:
        print(f"Error: {response.status_code}, Message: {response.text}")

    # Now you can pass this plant name to the generate_plant_message function


In [ ]:
plant_name_with_highest_probability = identify_plant(image_path)
print(plant_name_with_highest_probability)
PLANT_NAME = plant_name_with_highest_probability

Species: Rosa chinensis, Probability: 67.26%


Species: Rosa × odorata, Probability: 23.71%


Species: Rosa lucieae, Probability: 3.50%


Species: Rosa gallica, Probability: 2.31%


Plant with the highest probability: Rosa chinensis
Rosa chinensis


## Health Assessment

In [31]:

def get_health_assessment(image_path):
    # Define the endpoint and API key
    url = "https://api.plant.id/v2/health_assessment"
    #api_key = userdata.get('plantid_key')  # Replace with your actual API key
    api_key= "V3bNS9Yrc6yWsY2pgevJnjNoYtnhIw6Uvh9PxvuNwiyTyYf5Tr"

    # Load the image and encode it in base64
    image_path = "plant.jpg"
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode("utf-8")

    # Prepare headers and data payload
    headers = {
        "Content-Type": "application/json",
        "Api-Key": api_key
    }

    data = {
        "images": [base64_image],
        "organs": ["leaf"],  # Optional: Specify the plant part (e.g., leaf, flower, etc.)
        "include-related-images": False  # Optional: Include related images in the response
    }

    '''watering - dict - minimum and maximum of how wet environment the plant prefers 
    (1 = dry, 2 = medium, 3 = wet), eg. "watering": {"min": 1, "max": 2} 
    means plant prefers dry to medium environment

best_watering - string - short paragraph on optimal watering practices, English only

best_light_condition - string - short paragraph on optimal lighting, English only

best_soil_type - string - short paragraph on optimal soil types for growing the plant, English only'''

    # Send POST request
    response = requests.post(url, headers=headers, json=data)
    

    if response.status_code == 200:
        results = response.json()
        #print("Plant Health Assessment Results:")
        #print(results.keys())    
        # Check if the response contains health assessment information
        if "health_assessment" in results:
            health_info = results["health_assessment"]

            # Check if the plant is healthy
            if health_info["is_healthy"]:
                print(f"The plant is healthy with a probability of {health_info['is_healthy_probability']*100:.2f}%.")
                return health_info["is_healthy"]
            else:
                print("The plant shows signs of disease.")

                # If diseases are detected, print each disease and its details
                for disease in health_info["diseases"]:
                    pass
                    # print(f"Diagnosis: {disease['name']}")
                    # print(f"Probability: {disease['probability']*100:.2f}%")
                    # print(f"Suggested Treatment: {disease.get('treatment', 'N/A')}")
                    # print("\n")
                return health_info["diseases"]
        else:
            print("No health assessment information available in the response.")
    else:
        print(f"Error: {response.status_code}, Message: {response.text}")


### Wrapper for sending the health status to Groq

In [ ]:
import os
from groq import Groq
def send_groq_health_assessment(image_path, plant_name):
    """
    Sends plant health status message based on a picture and PlantID API to the Groq API.
    """

    # Initialize the Groq client
    client = Groq(
    api_key="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",
    )

    # Initialize the LINE bot with your channel access token
    # CHANNEL_ACCESS_TOKEN = os.getenv("CHANNEL_ACCESS_TOKEN")
    # bot = PlantitaBot(CHANNEL_ACCESS_TOKEN)
    #we have to remember to put these keys into the secrets
    bot = PlantitaBot("HYIfkCHmidAfgMc2KcXBLUXzpNvsvJ/t5k8VwU2+ekWovndds7h/5h2Qq+SZkoYjNCPuLg0BHANjAnX81xjbfT7imjJ1s+gYZf6XU4ttM4kuFTkChGSHDZUVKzFpTgesDxTWmILCYw0vgPz/bse7TQdB04t89/1O/w1cDnyilFU=")

    # get the health assessment of the plant
    health_status = get_health_assessment(image_path)
    health_status = health_status[0]

    #print("health assessment:", health_status)

    # craft the prompt
    user_message = (
        f"You are Plantita, an expert plant care advisor with a warm, caring personality like a concerned aunt.\n"
        f"Based on the health assessment of the plant {plant_name}:\n"
        f"{health_status}\n"
        f"Please analyze these conditions and provide:\n"
        "1. A caring, conversational assessment of the plant's current environment\n"
        "2. Specific recommendations for improvement if needed\n"
        "3. Any potential risks to the plant's health based on these conditions\n"
        "4. A simple action plan for the plant owner\n"

        "Keep your response friendly and encouraging, like a knowledgeable aunt giving advice about their beloved plants."
    )

    response = client.chat.completions.create(
        messages=[{"role": "user", "content": user_message}],
        model="llama3-8b-8192"
    )

    print("Plant health status message sent!")
    print(response.choices[0].message.content)

In [35]:
# Test
send_groq_health_assessment(image_path, PLANT_NAME)

The plant shows signs of disease.
health assessment: {'name': 'Leotiomycetes', 'probability': 0.1611, 'redundant': True, 'entity_id': 469, 'disease_details': {'local_name': 'Leotiomycetes', 'language': 'en'}}
Plant health status message sent!
Sweetie, I'm so glad you brought Rosa chinensis to me for a check-up! Let's take a closer look at the assessment together.

It seems like Leotiomycetes might be lurking in the background, with a probability of 0.1611. Now, don't worry just yet, but I do want to gently nudge you to take a closer look at the plant's environment. The fact that Leotiomycetes is detected tells me that the plant might be experiencing some stress, possibly due to factors like poor ventilation, inadequate light, or inconsistent watering.

Sweetheart, I think Rosa chinensis is craving a bit more TLC (tender loving care). Here are some super easy recommendations to get the plant back on track:

1. **Give it a little extra love**: Increase the air circulation around the plan

# Parse data and send it to Groq and then LINE

In [34]:
import random
import time
from datetime import datetime, timedelta



def simulate_sensor_data():
    """
    Simulates reading temperature, humidity, and light level data.
    """
    temperature = round(random.uniform(20.0, 30.0), 2)  # Celsius
    humidity = round(random.uniform(40.0, 60.0), 2)     # Percent
    moisture = round(random.uniform(30.0, 70.0), 2)  # Percent
    pressure = round(random.uniform(30.0, 70.0), 2)  # Percent

    return {"temperature": temperature, "humidity": humidity,"pressure": pressure, "moisture": moisture}

def send_groq_message(data):
    """
    Sends plant care message based on the data to the Groq API.
    """
    user_message = (
        f"You are Plantita, an expert plant care advisor with a warm, caring personality like a concerned aunt.\n"
        f"Current readings for Monstera Deliciosa:\n"
        f"    temperature={data['temperature']}°C,\n"
        f"    humidity={data['humidity']}%,\n"
        f"    pressure={data['pressure']}%,\n"
        f"    moisture={data['moisture']}%,\n\n"

        "Ideal conditions for this plant:\n"
         "- Temperature: [Fill in with the values according to the {plant_name}]\n"
        "- Humidity: [Fill in with the values according to the {plant_name}]\n"
        "- Pressure: [Fill in with the values according to the {plant_name}]\n"
        "- Moisture: [Fill in with the values according to the {plant_name}]\n\n"

        "Please analyze these conditions and provide:\n"
        "1. A caring, conversational assessment of the plant's current environment\n"
        "2. Specific recommendations for improvement if needed\n"
        "3. Any potential risks to the plant's health based on these conditions\n"
        "4. A simple action plan for the plant owner\n\n"
        "Keep your response friendly and encouraging, like a knowledgeable aunt giving advice about their beloved plants."
    )

    response = client.chat.completions.create(
        messages=[{"role": "user", "content": user_message}],
        model="llama3-8b-8192"
    )

    print("Plant care message sent!")
    print(response.choices[0].message.content)


In [35]:
import os
from groq import Groq

client = Groq(
    api_key="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",
)

# Initialize the LINE bot with your channel access token
# CHANNEL_ACCESS_TOKEN = os.getenv("CHANNEL_ACCESS_TOKEN")
# bot = PlantitaBot(CHANNEL_ACCESS_TOKEN)
#we have to remember to put these keys into the secrets
bot = PlantitaBot("HYIfkCHmidAfgMc2KcXBLUXzpNvsvJ/t5k8VwU2+ekWovndds7h/5h2Qq+SZkoYjNCPuLg0BHANjAnX81xjbfT7imjJ1s+gYZf6XU4ttM4kuFTkChGSHDZUVKzFpTgesDxTWmILCYw0vgPz/bse7TQdB04t89/1O/w1cDnyilFU=")


# Step 1: Generate the message from Groq
def generate_plant_message(temperature, humidity, pressure, moisture, plant_name):
    """
    Generates a plant care message using the Groq API.
    """
    user_message = (
        f"You are Plantita, an expert plant care advisor.\n"
        f"Current readings for {plant_name}:\n"
        f"    temperature={temperature}°C,\n"
        f"    humidity={humidity}%,\n"
        f"    pressure={pressure}%,\n"
        f"    moisture={moisture}%,\n\n"
        
        "Ideal conditions:\n"
        "- Temperature: [Fill in with the values according to the {plant_name}]\n"
        "- Humidity: [Fill in with the values according to the {plant_name}]\n"
        "- Pressure: [Fill in with the values according to the {plant_name}]\n"
        "- Moisture: [Fill in with the values according to the {plant_name}]\n\n"
        "Provide a friendly message for the user."
    )

    response = client.chat.completions.create(
        messages=[{"role": "user", "content": user_message}],
        model="llama3-8b-8192"
    )

    return response.choices[0].message.content


# Step 2: Broadcast or send the generated message
def send_plant_status_to_followers(temperature, humidity, pressure, moisture, plant_name):
    message = generate_plant_message(temperature, humidity, pressure, moisture,  plant_name)
    bot.broadcast_message(message)




# Simulated loop, since I don't have the Arduino with me now

In [ ]:
# Simulation loop
from time import sleep
from datetime import datetime, timedelta
# Set the interval for messaging (4 hours)
message_interval = timedelta(hours=4)
next_message_time = datetime.now()
PLANT_NAME = "Eustoma russellianum"  # Example plant name
try:
    while True:
        # Simulate sensor data readings
        data = simulate_sensor_data()
        print(f"Simulated Data: {data}")

        # Check if it's time to send a message
        if datetime.now() >= next_message_time:
            # Send plant status message to followers
            send_plant_status_to_followers(data['temperature'], data['humidity'], data["moisture"], data['moisture'], PLANT_NAME)
            # Set the next message time to four hours from now
            next_message_time = datetime.now() + message_interval

        # Wait before simulating the next data reading
        time.sleep(60)  # Adjust this to control how often data is simulated; 60 seconds is used here for demonstration

except KeyboardInterrupt:
    print("Simulation stopped.")

Simulated Data: {'temperature': 20.24, 'humidity': 43.91, 'pressure': 40.35, 'moisture': 31.95}


In [ ]:
import serial
import time
from datetime import datetime, timedelta

# Constants for the messaging interval
MESSAGE_INTERVAL = timedelta(seconds=4)
PLANT_NAME = "Your Plant"  # Replace with your plant's name
next_message_time = datetime.now()

# Serial port settings
SERIAL_PORT = 'COM5'  # Update to your correct serial port
BAUD_RATE = 9600  # Must match the Arduino's baud rate

# Open the serial connection
ser = serial.Serial(SERIAL_PORT, BAUD_RATE)
time.sleep(2)  # Allow time for the connection to establish

def parse_sensor_data(raw_data):
    """Parses sensor data from a formatted string."""
    sensor_data = {}
    lines = raw_data.strip().split('\n')  # Split by new lines
    
    try:
        for line in lines:
            if "Temperature" in line:
                sensor_data['temperature'] = float(line.split(":")[1].strip().split()[0])
            elif "Humidity" in line:
                sensor_data['humidity'] = float(line.split(":")[1].strip().split()[0])
            elif "Pressure" in line:
                sensor_data['pressure'] = float(line.split(":")[1].strip().split()[0])
            elif "Soil Moisture (Analog)" in line:
                sensor_data['soil_moisture_analog'] = float(line.split(":")[1].strip().split()[0])
            elif "Soil Moisture (Digital)" in line:
                sensor_data['soil_moisture_digital'] = line.split(":")[1].strip()
    except (IndexError, ValueError) as e:
        print(f"Error parsing line: {line}, Error: {e}")
    
    return sensor_data


print("Collecting data and monitoring plant status...")

try:
    buffer = ""
    while True:
        # Read and decode a line from the serial connection
        line = ser.readline().decode('utf-8').strip()
        
        if line == "---------------------------":
            # Process the accumulated data when a full block is received
            sensor_data = parse_sensor_data(buffer)
            buffer = ""  # Clear the buffer for the next block
            
            if sensor_data:
                #print(f"Real Sensor Data: {sensor_data}")
                
                # Check if it's time to send a message
                if datetime.now() >= next_message_time:
                    send_plant_status_to_followers(
                        sensor_data.get('temperature'),
                        sensor_data.get('humidity'),
                        sensor_data.get('pressure'),
                        sensor_data.get('soil_moisture_analog'),
                        PLANT_NAME
                    )
                    # Set the next message time
                    next_message_time = datetime.now() + MESSAGE_INTERVAL
        else:
            buffer += line + "\n"  # Accumulate lines into the buffer

        # Wait before reading the next line
        time.sleep(0.1)

except KeyboardInterrupt:
    print("Monitoring stopped.")

finally:
    ser.close()  # Always close the serial connection


Monitoring stopped.
